## Loading of modules and packages

Activate the Julia environment of the project.

In [ ]:
using Pkg
Pkg.activate("..\\")

Load the necessary packages.

In [ ]:
using ArchGDAL
using GeoArrays
using GeoStats
using Plots

Load the necessary modules.

In [ ]:
include("..\\src\\Data Gathering\\GroundData.jl")
include("..\\src\\Data Gathering\\SatelliteData.jl")

include("..\\src\\Analysis\\Utils\\Functions.jl")

include("..\\src\\Analysis\\Aquifers.jl")
include("..\\src\\Analysis\\Lakes.jl")
include("..\\src\\Analysis\\Noises.jl")
include("..\\src\\Analysis\\Plumes.jl")
include("..\\src\\Analysis\\Sediments.jl")


Aliasing of the modules to shorten the code.

In [ ]:
# Packages
const agd = ArchGDAL
const ga = GeoArrays

# Data gathering
const grdt = GroundData
const stdt = SatelliteData

## Code testing 

#### Data Gathering testing

In [ ]:
# DataFrame containing the gathered data, DataFrame containing the data that is not geolocalized. 
resmt, mresmt = grdt.getGroundData(:METEO, grdt.AA, grdt.FVG, grdt.L, grdt.T, grdt.V)

In [ ]:
# Parameters measured by the stations
unique(resmt.parameter)

In [ ]:
# All relative humidity measurements
measurements = resmt[ resmt.parameter .== "Umidità relativa", : ]

# All the measurement's values
values = ( val = measurements.value, )

# All the coordinates of the measurements
coords = Tuple{Float64, Float64}[ (c[1], c[2]) for c in eachrow(measurements[:, [:longitude, :latitude]]) ]

In [ ]:
dtm_file = "..\\resources\\Analysis data\\DTM_32.tiff"
dtm = agd.read(dtm_file)

In [ ]:
source_file = "..\\resources\\Analysis data\\source_shapefile\\source_32.shp"
source = agd.read(source_file)
src_geom = agd.getgeom(collect(agd.getlayer(source, 0))[1])

In [ ]:
c1, c2 = Functions.toCoords.( Ref(dtm), [(1, 0), (2, 0)] )
c2[1] - c1[1]

In [ ]:
#= Based on:
    https://juliaearth.github.io/GeoStats.jl/stable/
=#

# Georeference data
D = georef(values, coords)

# Estimation domain
#G = CartesianGrid( (100, 100), src_geom, )
G = CartesianGrid( (100, 100) )

# Estimation problem
problem = EstimationProblem(D, G, :val)

# Solver from the list of solvers
solver = Kriging( :val => ( variogram = GaussianVariogram(range=35.0), ) )

# Solving problem
solution = solve(problem, solver)

# Solution plot
contourf(solution, clabels=true)

#### Data Analysis testing

##### Initial Setup

In [ ]:
# Digital Terrain Model, the main raster for the analysis, reppresents Veneto
dtm_file = "..\\resources\\Analysis data\\DTM_32.tiff"
dtm = agd.read(dtm_file)

In [ ]:
# Test impedence raster, contains data on the acoustic impedence of terrain
impedence_file = "..\\resources\\Analysis data\\impedances.tiff"
impd = agd.read(impedence_file)

Test area for aquifers and lakes

In [ ]:
# Shapefile holding the poligon reppresenting the surface of a lake or the
 # limits of an quifer
area_file = "..\\resources\\Analysis data\\area\\area.shp"
area = agd.read(area_file)

Create a source point to test the analysis functions.

In [ ]:
# Coordinates of the source
#   lat, lon = (11.930065824163105, 45.425861311724816) # WGS84
lat, lon = (726454.9302346368, 5025993.899219433)
# Path to the source shapefile
source_dir = "..\\resources\\Analysis data\\source_shapefile"
# Directory holding the `source` files
!isdir(source_dir) && mkdir(source_dir)
# Creation of a shapefile containing the point if not already existing
if "source_32.shp" ∉ readdir(source_dir)
    agd.create(
        source_dir*"\\source_32.shp",
        driver = agd.getdriver("ESRI Shapefile")
    ) do ds
        agd.createlayer(
            geom = agd.wkbPoint,
            # spatialref = agd.importEPSG(4326) # WGS84
            spatialref = agd.importEPSG(32632)
        ) do layer
            agd.createfeature(layer) do feature
                agd.setgeom!(feature, agd.createpoint(lat, lon))
            end
            agd.copy(layer, dataset=ds)
        end
    end
end
source_file = source_dir*"\\source_32.shp"
source = agd.read(source_file)

In [ ]:
src_geom = agd.getgeom(collect(agd.getlayer(source, 0))[1])

#### Analysis functions execution.

##### Pollutants diffusion in an aquifer

In [ ]:
Aquifers.run_aquifer(
    "..\\resources\\Analysis results\\test_aquifer.tiff",
    dtm_file,
    source_file,
    area_file,
    "108-88-3",
    100.0,
    1000.0,
    45,
    20.0,
    "sand",
    tolerance = 2,
    time = 10,
    orthogonal_width = 10.0,
    mixing_zone_depth = 1580.0,
    algorithm = :domenico
)

In [ ]:
# Remove the source point value
rast = ga.read("..\\resources\\Analysis results\\test_aquifer.tiff")
replace!( rast, maximum(skipmissing(rast.A)) => missing )
#Plot the results of the analysis
heatmap(
    rast,
    plot_title = "Aquifer pollution diffusion",
    xlabel = "X coordinate (m)",
    ylabel = "Y coordinate (m)",
    color = :cividis
)
#Add the source
plot!(src_geom)

<br> 

##### Polutants dispersion in lakes

In [ ]:
Lakes.run_lake(
    "..\\resources\\Analysis results\\test_lake.tiff",
    dtm_file,
    source_file,
    area_file,
    2000.0,
    135,
    0.03,
    10.0,
    tolerance = 2,
    fickian_x = 4.0,
    fickian_y = 3.0
)

In [ ]:
# Remove the source point value
rast = ga.read("..\\resources\\Analysis results\\test_lake.tiff")
replace!( rast, maximum(skipmissing(rast.A)) => missing )
#Plot the results of the analysis
heatmap(
    rast,
    plot_title = "Lake pollution diffusion",
    xlabel = "X coordinate (m)",
    ylabel = "Y coordinate (m)",
    color = :inferno
)
#Add the source
plot!(src_geom)

<br>

##### Noise pollution

In [ ]:
Noises.run_noise(
    "..\\resources\\Analysis results\\test_noise.tiff",
    dtm_file,
    impedence_file,
    source_file,
    293.15,
    0.2,
    110.0,
    400.0,
)

In [ ]:
# Remove the source point value
rast = ga.read("..\\resources\\Analysis results\\test_noise.tiff")
replace!( rast, maximum(skipmissing(rast.A)) => missing )
#Plot the results of the analysis
heatmap(
    rast,
    plot_title = "Noise pollution",
    xlabel = "X coordinate (m)",
    ylabel = "Y coordinate (m)",
    colorbar_title = "Noise level (dB S.P.L.)",
    color = :winter
)
#Add the source
plot!(src_geom)

<br>

##### Airborne pollutants dispersion from a stack

In [ ]:
Plumes.run_plume(
    "..\\resources\\Analysis results\\test_plume.tiff",
    dtm_file,
    source_file,
    "a",
    "c",
    10000.0,
    225,
    0.1,
    80.0,
    1.0,
    tolerance = 2,
    gas_velocity = 0.1,
    gas_temperature = 150.0,
    temperature = 18.0
)

In [ ]:
# Remove the source point value
rast = ga.read("..\\resources\\Analysis results\\test_plume.tiff")
replace!( rast, maximum(skipmissing(rast.A)) => missing )
#Plot the results of the analysis
heatmap(
    rast,
    plot_title = "Plume pollution diffusion",
    xlabel = "X coordinate (m)",
    ylabel = "Y coordinate (m)",
    colorbar_title = "Substance concentration",
    color = :seaborn_rocket_gradient
)
#Add the source
plot!(src_geom)

<br>

##### Pollutants sedimentation

In [ ]:
Sediments.run_sediment(
    "..\\resources\\Analysis results\\test_sediment.tiff",
    dtm_file,
    source_file,
    0.03,
    13.0,
    1.0,
    10.0,
    4.0,
    315,
    0.0359,
    100,
    1,
    tolerance = 2
)

In [ ]:
# Remove the source point value
rast = ga.read("..\\resources\\Analysis results\\test_sediment.tiff")
replace!( rast, maximum(skipmissing(rast.A)) => missing )
#Plot the results of the analysis
heatmap(
    rast,
    plot_title = "Sediment pollution diffusion",
    xlabel = "X coordinate (m)",
    ylabel = "Y coordinate (m)",
    color = :viridis
)
#Add the source
plot!(src_geom)